# Measuring the Gender Pay Gap with Blinder-Oaxaca Decompositions
---------
#### Chris Fong and Ronald Walker

Due May 9th at 5pm

Requirements:

1. It should involve the use of real data to explore a question of interest to you.

2. The data should be analyzed using figures/maps, summary tables as well as statistical methods introduced in Ec 140, 141 or C142. As a rough guide a good project will include 2-3 figures, 2-3 displays of summary or descriptive statistics (e.g., cross-tabs) and the presentation of estimation results from a small collection of econometric models (e.g., OLS or quantile regression fits with different conditioning variables).

3. The analysis and write-up should be done in Python using a Jupyter notebook (with all project narrative included in markdown cells). Code should be clearly commented. The write-up should explain the assumptions being invoked to justify the analysis and also include a discussion and interpretation of any results. A clear discussion of the limitations of the analysis and how it might be improved through further work should also be included.

4. group projects need to include a few paragraphs explaining each team members specific contributions

0 -- Background

1 -- Implementation and Interpretation of Blinder-Oaxaca

2 -- Data Collection and Cleaning

3 -- Application of Blinder-Oaxaca

## 0. Background

Motivation -- may merge w/ 1.

## 1. Implementation and Interpretation of Blinder-Oaxaca

Implement Blinder-Oaxaca using statsmodels

## 2. Data

Some data set

## 3. Application

For each country of interest:

Current measure of pay gap inequality
Standardized samples of earnings/background census information
Regress log(wages) on gender and other variables (race, number of dependents, education, etc.) and control for role/industry, industry, neither

Plot current measure of inequality vs unexplained proportion from B-O